### Cleaning and merging data for the knn tutorial

##### NOTE: I originally planned to use data from 2014-2017, but instead opted to just use 2017 data. I've retained all the code for 14-17 in case I wanted to use it in a later project.

Data Sources:

State Population Data: https://www.census.gov/data/datasets/time-series/demo/popest/2010s-state-total.html

Infant Mortality Data: https://www.cdc.gov/nchs/pressroom/sosmap/infant_mortality_rates/infant_mortality.htm

NASA Data: https://prod.nais.nasa.gov/cgibin/npdv/npdv.cgi

House Subcommittee: https://en.wikipedia.org/wiki/United_States_House_Science_Subcommittee_on_Space_and_Aeronautics#115th_Congress

Senate Subcommittee: 
https://en.wikipedia.org/wiki/United_States_Senate_Commerce_Subcommittee_on_Space,_Science,_and_Competitiveness

In [3]:
## Import Necessary Libraries

import pandas as pd
import numpy as np

In [4]:
## Import NASA contract data and restrict to 2014-2017

NASAdata = pd.read_csv("NASA_2005-2019_by_State.csv")
NASAdata.head()

,State,Year,Award
0,AK,2005,0.151263
1,AK,2006,0.141344
2,AK,2007,0.129998
3,AK,2008,0.180237
4,AK,2009,0.146615


In [5]:
## Import Infant Mortality Data

InfMort = pd.read_csv("Infant_Mortality_Rates_by_State_2014-2017.csv")
InfMort = InfMort.dropna()
InfMort["State"] = InfMort["STATE"]
InfMort["Year"] = InfMort.YEAR.astype(np.int64)
InfMort["Year"] = InfMort.Year.astype(str)
InfMort = InfMort.drop(["URL","YEAR","STATE"],axis=1)
InfMort.head(20)

,RATE,DEATHS,State,Year
0,7.4,434,AL,2017
1,5.6,59,AK,2017
2,5.7,469,AZ,2017
3,8.2,307,AR,2017
4,4.2,"1,973",CA,2017
5,4.5,291,CO,2017
6,4.5,160,CT,2017
7,6.6,72,DE,2017
8,6.1,"1,358",FL,2017
9,7.2,932,GA,2017


In [6]:
type(InfMort.Year[5])

str

In [7]:
## Data dictionary of states and abbreviations for state
states = {'AK': 'Alaska','AL': 'Alabama','AR': 'Arkansas','AS': 'American Samoa','AZ': 'Arizona','CA': 'California',
        'CO': 'Colorado','CT': 'Connecticut','DC': 'District of Columbia','DE': 'Delaware','FL': 'Florida',
        'GA': 'Georgia','GU': 'Guam','HI': 'Hawaii','IA': 'Iowa','ID': 'Idaho','IL': 'Illinois','IN': 'Indiana',
        'KS': 'Kansas','KY': 'Kentucky','LA': 'Louisiana','MA': 'Massachusetts','MD': 'Maryland','ME': 'Maine',
        'MI': 'Michigan','MN': 'Minnesota','MO': 'Missouri','MP': 'Northern Mariana Islands','MS': 'Mississippi',
        'MT': 'Montana','NA': 'National','NC': 'North Carolina','ND': 'North Dakota','NE': 'Nebraska',
        'NH': 'New Hampshire','NJ': 'New Jersey','NM': 'New Mexico','NV': 'Nevada','NY': 'New York','OH': 'Ohio',
        'OK': 'Oklahoma','OR': 'Oregon','PA': 'Pennsylvania','PR': 'Puerto Rico','RI': 'Rhode Island',
        'SC': 'South Carolina','SD': 'South Dakota','TN': 'Tennessee','TX': 'Texas','UT': 'Utah','VA': 'Virginia',
        'VI': 'Virgin Islands','VT': 'Vermont','WA': 'Washington','WI': 'Wisconsin','WV': 'West Virginia',
        'WY': 'Wyoming'}

Newstates = dict([(value, key) for key, value in states.items()]) 
Newstates["Alaska"]

'AK'

In [8]:
## Import population data
StatePop = pd.read_csv("State_Pop_2010-2017.csv")

## Create State Name column (State column should just be abbreviations)
StatePop["State Name"] = [x[1:] for x in StatePop.State]

## Drop unnecessary columns
StatePop = StatePop.drop(["State","Census","Basis","2010","2011","2012","2013"],axis=1)

## Melt data from wide to long
StatePop = pd.melt(StatePop,id_vars=["State Name"],var_name="Year",value_name="Population")

## Use the Newstates dictionary to create column of State Abbreviations.
StatePop["State"] = StatePop["State Name"].map(Newstates) 
StatePop.head()

,State Name,Year,Population,State
0,Alabama,2014,"4,842,481",AL
1,Alaska,2014,"736,307",AK
2,Arizona,2014,"6,733,840",AZ
3,Arkansas,2014,"2,967,726",AR
4,California,2014,"38,625,139",CA


In [9]:
## Segment out our 2014-2017 data
NASAdata = NASAdata[NASAdata.Year.isin([2014,2015,2016,2017])]
NASAdata["Year"] = NASAdata.Year.astype(str)
NASAdata = NASAdata.reset_index(drop=True)
NASAdata.head()

,State,Year,Award
0,AK,2014,0.162813
1,AK,2015,0.119287
2,AK,2016,0.129675
3,AK,2017,0.090561
4,AL,2014,13.200416


In [10]:
type(InfMort.Year[1])

str

In [11]:
## Merge data sets

data = NASAdata.merge(StatePop)
data = data.merge(InfMort)
data = data[['State Name','State', 'Year', 'Award', 'Population', 'RATE', 'DEATHS']]
data.head()

,State Name,State,Year,Award,Population,RATE,DEATHS
0,Alaska,AK,2014,0.162813,"736,307",6.6,75
1,Alaska,AK,2015,0.119287,"737,547",6.9,78
2,Alaska,AK,2016,0.129675,"741,504",5.4,60
3,Alaska,AK,2017,0.090561,"739,786",5.6,59
4,Alabama,AL,2014,13.200416,"4,842,481",8.7,516


array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
       'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI',
       'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV',
       'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
       'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

In [12]:
CenterDict = dict({'GSFC':'Goddard Space Flight Center, MD',
                   'JSC':'Johnson Space Center, TX',
                   'KSC':'Kennedy Space Center, FL',
                   'MSFC':'Marshall Space Flight Center, AL',
                   'NMO':'Jet Propulsion Laboratory, CA',
                   'AFRC':'Armstrong Flight Research Center, CA',
                   'APL':'Applied Physics Lab, MD',
                   'ARC':'Ames Research Center, CA',
                   'GRC':'Glenn Research Center, OH', 
                   'HQ':'NASA Headquarters, DC', 
                   'LaRC':'Langley Research Center, VA',
                   'NSSC':'NASA Shared Services Center, MS',
                   'SSC':'Stennis Space Center, MS'})
CenterStates = ["MD","TX","FL","AL","CA","OH","VA","MS"]

In [13]:
data["Has_Facility"] = np.where(data.State.isin(CenterStates),1,0)
data.head()

,State Name,State,Year,Award,Population,RATE,DEATHS,Has_Facility
0,Alaska,AK,2014,0.162813,"736,307",6.6,75,0
1,Alaska,AK,2015,0.119287,"737,547",6.9,78,0
2,Alaska,AK,2016,0.129675,"741,504",5.4,60,0
3,Alaska,AK,2017,0.090561,"739,786",5.6,59,0
4,Alabama,AL,2014,13.200416,"4,842,481",8.7,516,1


In [14]:
data["State-Year"] = data.State +"_" + data.Year
data.head()

,State Name,State,Year,Award,Population,RATE,DEATHS,Has_Facility,State-Year
0,Alaska,AK,2014,0.162813,"736,307",6.6,75,0,AK_2014
1,Alaska,AK,2015,0.119287,"737,547",6.9,78,0,AK_2015
2,Alaska,AK,2016,0.129675,"741,504",5.4,60,0,AK_2016
3,Alaska,AK,2017,0.090561,"739,786",5.6,59,0,AK_2017
4,Alabama,AL,2014,13.200416,"4,842,481",8.7,516,1,AL_2014


In [29]:
## Convert population strings to floats
data["Population"] = [float(str(x).replace(",", "")) for x in data["Population"]]
data["Rate"] = [float(str(x).replace(",", "")) for x in data["RATE"]]
data['Deaths'] = [float(str(x).replace(",", "")) for x in data["DEATHS"]]
data.head()

,State Name,State,Year,Award,Population,RATE,DEATHS,Has_Facility,State-Year,Rate,Deaths
0,Alaska,AK,2014,0.162813,736307.0,6.6,75,0,AK_2014,6.6,75.0
1,Alaska,AK,2015,0.119287,737547.0,6.9,78,0,AK_2015,6.9,78.0
2,Alaska,AK,2016,0.129675,741504.0,5.4,60,0,AK_2016,5.4,60.0
3,Alaska,AK,2017,0.090561,739786.0,5.6,59,0,AK_2017,5.6,59.0
4,Alabama,AL,2014,13.200416,4842481.0,8.7,516,1,AL_2014,8.7,516.0


In [32]:
data = data.drop(['RATE',"DEATHS"],axis=1)
data.head()

,State Name,State,Year,Award,Population,Has_Facility,State-Year,Rate,Deaths
0,Alaska,AK,2014,0.162813,736307.0,0,AK_2014,6.6,75.0
1,Alaska,AK,2015,0.119287,737547.0,0,AK_2015,6.9,78.0
2,Alaska,AK,2016,0.129675,741504.0,0,AK_2016,5.4,60.0
3,Alaska,AK,2017,0.090561,739786.0,0,AK_2017,5.6,59.0
4,Alabama,AL,2014,13.200416,4842481.0,1,AL_2014,8.7,516.0


In [37]:
## Subset 2017 data
data17 = data[data.Year == "2017"].reset_index(drop=True)
data17.head()

,State Name,State,Year,Award,Population,Has_Facility,State-Year,Rate,Deaths
0,Alaska,AK,2017,0.090561,739786.0,0,AK_2017,5.6,59.0
1,Alabama,AL,2017,12.687506,4875120.0,1,AL_2017,7.4,434.0
2,Arkansas,AR,2017,0.012468,3002997.0,0,AR_2017,8.2,307.0
3,Arizona,AZ,2017,1.674364,7048876.0,0,AZ_2017,5.7,469.0
4,California,CA,2017,39.838185,39399349.0,1,CA_2017,4.2,1973.0


These will be easiest to just manually add in Excel. I also manually added a binary for top ten and top five contract awards. 

115th Congress members on House Science 
Subcommittee on Space and Aeronautics
AL:1
AZ:1
CA:4
CO:1
FL:4
IN:1
IL:2
LA:2
OK:1
PA:1
TX:1
VA:2


115th Senate subcommittee:
AK
CO
FL
HI
KA
MA
MI
NH
NM
SD
TX
UT
WV
WIx2

In [38]:
data17.to_csv("Cleaned_2017_NASA_POP_INFMORT.csv",index=False)

In [40]:
newdata = pd.read_csv("Cleaned_2017_NASA_POP_INFMORT.csv")
newdata["Logged_pop"] = [np.log(x) for x in newdata["Population"]]
newdata.head()

,State-Year,State Name,State,Year,Award,Population,Has_Facility,Rate,Deaths,Senate,House,Top_Ten,Top_Five,Logged_pop
0,AK_2017,Alaska,AK,2017,0.090561,739786,0,5.6,59,1,0,0,0,13.514116
1,AL_2017,Alabama,AL,2017,12.687506,4875120,1,7.4,434,0,1,1,1,15.399655
2,AR_2017,Arkansas,AR,2017,0.012468,3002997,0,8.2,307,0,0,0,0,14.915121
3,AZ_2017,Arizona,AZ,2017,1.674364,7048876,0,5.7,469,0,1,0,0,15.768379
4,CA_2017,California,CA,2017,39.838185,39399349,1,4.2,1973,0,4,1,1,17.489260


In [41]:
newdata.to_csv("Cleaned_2017_NASA_POP_INFMORT.csv",index=False)